<a href="https://colab.research.google.com/github/himoyuzuki/colab_analytics/blob/main/OpenStreetMap%E3%81%8B%E3%82%89%E3%82%AB%E3%83%95%E3%82%A7%E3%81%AE%E5%88%86%E5%B8%83%E3%82%92%E5%88%86%E6%9E%90%E3%81%99%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenStreetMapからカフェの分布を分析する

## 下準備

In [ ]:
!pip install osmnx geopandas folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 2.5 MB/s eta 0:00:00


In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
from functools import reduce
import pandas as pd
import geopandas as gpd
import osmnx as ox
import folium
import plotly.express as px
from scipy import stats

## 札幌市のカフェを可視化

In [ ]:
# 札幌市のカフェ情報をGeoPandas DataFrame形式で取得
place_name = "Sapporo, Hokkaido, Japan"
tags = {"amenity": "cafe"}
cafes_gdf = ox.features.features_from_place(place_name, tags)

In [ ]:
cafes_gdf.head()

geometry addr:block_number addr:city  \
element id                                                                   
node    1554997456  POINT (141.29911 43.04446)                20       札幌市   
        1669503978  POINT (141.34702 43.06523)                 1       札幌市   
        1669503980  POINT (141.34666 43.06468)               NaN       札幌市   
        1669736660   POINT (141.3394 43.05745)                 4       札幌市   
        1679095442  POINT (141.33922 43.05702)               NaN       NaN   

                   addr:housenumber addr:neighbourhood addr:postcode  \
element id                                                             
node    1554997456                7                2丁目      064-0944   
        1669503978                3            北4条西6丁目      060-0004   
        1669503980              NaN            北3条西6丁目      060-0003   
        1669736660              182           南1条西12丁目      060-0061   
        1679095442              NaN                NaN           NaN   

                   addr:province addr:quarter addr:suburb amenity  \
element id                                                          
node    1554997456           北海道         円山西町         中央区    cafe   
        1669503978           北海道          NaN         中央区    cafe   
        1669503980           北海道          NaN         中央区    cafe   
        1669736660           北海道          NaN         中央区    cafe   
        1679095442           NaN          NaN         NaN    cafe   

                            name             name:en       name:ja  \
element id                                                           
node    1554997456         アダージョ              Adajio         アダージョ   
        1669503978      純喫茶オリンピア   JUN-KISSA Olympia      純喫茶オリンピア   
        1669503980  ドトールコーヒーショップ  Doutor Coffee Shop  ドトールコーヒーショップ   
        1669736660     アトリエ・モリヒコ    Atelier Morihiko     アトリエ・モリヒコ   
        1679095442           NaN                 NaN           NaN   

                   reservation                                 website  \
element id                                                               
node    1554997456    required                   http://www.adajio.jp/   
        1669503978         NaN          https://www.olympia-coffee.jp/   
        1669503980         NaN               https://www.doutor.co.jp/   
        1669736660         NaN  https://www.morihico.com/shop/atelier/   
        1679095442         NaN                                     NaN   

                   air_conditioning      cuisine indoor_seating  \
element id                                                        
node    1554997456              NaN          NaN            NaN   
        1669503978              yes  coffee_shop            yes   
        1669503980              NaN  coffee_shop            yes   
        1669736660              NaN  coffee_shop            NaN   
        1679095442              NaN          NaN            NaN   

                   internet_access internet_access:fee level  \
element id                                                     
node    1554997456             NaN                 NaN   NaN   
        1669503978            wlan           customers    -1   
        1669503980             NaN                 NaN     0   
        1669736660             NaN                 NaN   NaN   
        1679095442             NaN                 NaN   NaN   

                                           opening_hours outdoor_seating  \
element id                                                                 
node    1554997456                                   NaN             NaN   
        1669503978                     Mo-Fr 10:00-18:00              no   
        1669503980                     Mo-Fr 08:00-18:30              no   
        1669736660  Mo-Fr 08:00-22:00, Sa-Su 11:00-22:00             NaN   
        1679095442                                   NaN             NaN   

                   payment:cash payment:

In [ ]:
# 地図にピンを立てて可視化
gdf_points = cafes_gdf[cafes_gdf.geometry.apply(lambda x: x.geom_type == 'Point')]

center_lat = gdf_points.geometry.y.mean()
center_lon = gdf_points.geometry.x.mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

for idx, row in gdf_points.iterrows():  # マーカーの追加
    folium.Marker([row.geometry.y, row.geometry.x], popup=row.get('name', 'Cafe')).add_to(m)

display(m)

## 札幌市の駅とカフェの関係を分析

In [ ]:
# 札幌市の駅を取得
tags = {"railway": "station"}
stations_gdf = ox.features.features_from_place(place_name, tags)

In [ ]:
# CRSへの変換。epsgは平面直角座標系の札幌に該当するエリアを選択
cafes_gdf_projected = cafes_gdf.to_crs(epsg=6680)
stations_gdf_points_projected = stations_gdf.to_crs(epsg=6680)

distances = []
nearest_station_names = []

# すべてのカフェについて処理を繰り返す
for index, cafe_row in cafes_gdf_projected.iterrows():
    # 最寄り駅を算出（stations_gdfのgeometry列とcafe_rowのgeometryを比較）
    nearest_station_index = stations_gdf_points_projected.geometry.distance(cafe_row.geometry).idxmin()

    # 最寄り駅のGeoDataFrameを取得
    nearest_station_row = stations_gdf_points_projected.loc[nearest_station_index]

    # 距離を計算
    distance = cafe_row.geometry.distance(nearest_station_row.geometry)
    distances.append(distance)

    # 最寄り駅の名前を収集
    nearest_station_name = stations_gdf.loc[nearest_station_row.name]['name']
    nearest_station_names.append(nearest_station_name)


# cafes_gdf GeoDataFrameに距離情報を追加
cafes_gdf['distance_to_nearest_station'] = distances
cafes_gdf['nearest_station_name'] = nearest_station_names

cafes_gdf[['name', 'nearest_station_name', 'distance_to_nearest_station']].head()

name nearest_station_name  \
element id                                              
node    1554997456         アダージョ                 円山公園   
        1669503978      純喫茶オリンピア                 さっぽろ   
        1669503980  ドトールコーヒーショップ                 さっぽろ   
        1669736660     アトリエ・モリヒコ                西11丁目   
        1679095442           NaN                西11丁目   

                    distance_to_nearest_station  
element id                                       
node    1554997456                  2078.904513  
        1669503978                   309.802491  
        1669503980                   357.908473  
        1669736660                   214.515722  
        1679095442                   258.314084

In [ ]:
# 最寄り駅からの距離分布についてヒストグラムを作成して表示
fig = px.histogram(
        cafes_gdf,
        x='distance_to_nearest_station',
        nbins=50,
        title='札幌市内のカフェの最寄り駅からの距離分布'
    )
fig.show()